In [13]:
import pandas as pd
import sklearn.metrics as metrics

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [14]:
# Load weather data for Kursk
months = ['May', 'Jun', 'Jul', 'Aug', 'Sep']
varnames = ["temp", "rain", "cloud", "humid", "sun"]
vardict = {}
for varname in varnames:
    var = pd.read_csv(
        "./weather/" + varname + "/34009.txt",
        sep=r'\s+',
        names=['city', 'year'] + months
    )
    var = var.drop('city', axis=1)
    var[months] = var[months].astype(float)
    var['year'] = var['year'].astype(int)
    #var = var.set_index('year')
    var = var.groupby(['year']).max()
    # Calculate integral metrics for a year (over May - September)
    var = var.agg(['sum', 'mean'], axis=1)
    vardict[varname] = var

# Merge weather data into single dataframe
var = pd.DataFrame(var.index)
for varname in varnames:
    var = pd.merge(var, vardict[varname], how='right', on='year', suffixes=(None, f"_{varname}"))
var = var.set_index('year')

# Make sure labels are correct
columns = []
for varname in varnames:
    columns.append("sum_" + varname)
    columns.append("mean_" + varname)
var = var.set_axis(columns, axis=1)

var.head()

,sum_temp,mean_temp,sum_rain,mean_rain,sum_cloud,mean_cloud,sum_humid,mean_humid,sum_sun,mean_sun
year,,,,,,,,,,
2015,92.1,18.42,278.5,55.70,26.58,5.316,301.0,60.2,1445.0,289.0
2016,88.4,17.68,435.6,87.12,29.48,5.896,344.0,68.8,1350.0,270.0
2017,84.8,16.96,253.5,50.70,29.25,5.850,297.0,59.4,1435.0,287.0
2018,94.9,18.98,280.7,56.14,25.82,5.164,272.0,54.4,1556.0,311.2
2019,90.7,18.14,218.5,43.70,29.36,5.872,321.0,64.2,1465.0,293.0


In [15]:
# Load crops yield data for Kursk
years_todrop = [f"{i}" for i in range(2015,2018)]
targ = pd.read_csv("phenotypes_adjusted.tsv", sep="\t")
targ = targ.drop(years_todrop, axis=1)
# Unpivot dataframe over all years (to have single year column)
targ = targ.melt(id_vars=["sample"], var_name="year", value_name="yield")
# Set year as index
targ['sample'] = targ['sample'].astype(str)
targ['yield'] = targ['yield'].astype(float)
targ['year'] = targ['year'].astype(int)
targ = targ.set_index('year')
# Normalize yields?
#targets['yield'] = targets['yield'] / targets['yield'].max()
targ.head()

,sample,yield
year,,
2019,PS000196,99.0
2019,PS000195,100.0
2019,PS000121,107.0
2019,PS000126,115.0
2019,PS000123,113.0


In [16]:
data = pd.merge(targ, var, on='year')
data.head()

,sample,yield,sum_temp,mean_temp,sum_rain,mean_rain,sum_cloud,mean_cloud,sum_humid,mean_humid,sum_sun,mean_sun
year,,,,,,,,,,,,
2019,PS000196,99.0,90.7,18.14,218.5,43.7,29.36,5.872,321.0,64.2,1465.0,293.0
2019,PS000195,100.0,90.7,18.14,218.5,43.7,29.36,5.872,321.0,64.2,1465.0,293.0
2019,PS000121,107.0,90.7,18.14,218.5,43.7,29.36,5.872,321.0,64.2,1465.0,293.0
2019,PS000126,115.0,90.7,18.14,218.5,43.7,29.36,5.872,321.0,64.2,1465.0,293.0
2019,PS000123,113.0,90.7,18.14,218.5,43.7,29.36,5.872,321.0,64.2,1465.0,293.0


In [23]:
data.to_csv(weather_data, sep='\t')

,sample,yield,sum_temp,mean_temp,sum_rain,mean_rain,sum_cloud,mean_cloud,sum_humid,mean_humid,sum_sun,mean_sun
year,,,,,,,,,,,,
2019,PS000196,99.0,90.7,18.14,218.5,43.7,29.36,5.872,321.0,64.2,1465.0,293.0
2019,PS000195,100.0,90.7,18.14,218.5,43.7,29.36,5.872,321.0,64.2,1465.0,293.0
2019,PS000121,107.0,90.7,18.14,218.5,43.7,29.36,5.872,321.0,64.2,1465.0,293.0
2019,PS000126,115.0,90.7,18.14,218.5,43.7,29.36,5.872,321.0,64.2,1465.0,293.0
2019,PS000123,113.0,90.7,18.14,218.5,43.7,29.36,5.872,321.0,64.2,1465.0,293.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2019,PS000370,104.0,90.7,18.14,218.5,43.7,29.36,5.872,321.0,64.2,1465.0,293.0
2019,PS000184,95.0,90.7,18.14,218.5,43.7,29.36,5.872,321.0,64.2,1465.0,293.0
2019,PS000169,120.0,90.7,18.14,218.5,43.7,29.36,5.872,321.0,64.2,1465.0,293.0


In [ ]:
for year in range(2019, 2024)

In [22]:
# Initialize a dictionary to store coefficients for each sample
sample_coeffs = {}
# Loop over each sample group
for year in range(2019, 2024)
    # Features and target for the sample-year group
    sample_group = data.loc[year]
    x = sample_group[columns]
    y = sample_group['yield']
    # Scale and normalize data
    scaler = StandardScaler()
    x = scaler.fit_transform(x)
    # Fit a linear regression model
    model = LinearRegression()
    model.fit(x, y)
    # Store the coefficients for the given sample and year
    sample_coeffs[sample] = model.coef_
    # Scores
    mae = metrics.mean_absolute_error(model.predict(x), y)
    mse = metrics.mean_squared_error(model.predict(x), y)
    rmse = mse ** 0.5
    r2 = metrics.r2_score(model.predict(x), y)
print(f"MAE: {mae}, RMSE: {rmse}, R2: {r2}")
df = pd.DataFrame(sample_coeffs)
df.head()

MAE: 8.526512829121202e-15, RMSE: 1.1007680729470101e-14, R2: 1.0


,PS000026,PS000027,PS000028,PS000031,PS000033,PS000034,PS000047,PS000048,PS000050,PS000051,...,PS000433,PS000436,PS000437,PS000440,PS000441,PS000444,PS000566,PS000567,PS000568,PS000570
0,2.537319,2.118718,1.912452,-0.154924,3.516477,4.763922,0.711031,2.570694,4.250409,2.283641,...,2.194551,3.497283,8.204310,2.708987,2.325189,1.274560,-0.067503,-2.583769,0.0,-0.620724
1,2.537319,2.118718,1.912452,-0.154924,3.516477,4.763922,0.711031,2.570694,4.250409,2.283641,...,2.194551,3.497283,8.204310,2.708987,2.325189,1.274560,-0.067503,-2.583769,0.0,-0.620724
2,-0.023460,2.388284,5.111621,2.157594,3.000659,1.394094,1.423617,-0.061804,3.235104,1.858742,...,1.709888,1.765696,6.417299,1.701803,3.567923,2.546026,1.121828,-0.100628,0.0,-0.259503
3,-0.023460,2.388284,5.111621,2.157594,3.000659,1.394094,1.423617,-0.061804,3.235104,1.858742,...,1.709888,1.765696,6.417299,1.701803,3.567923,2.546026,1.121828,-0.100628,0.0,-0.259503
4,0.783746,2.283415,-0.365763,0.882981,1.412185,0.648176,0.506269,1.977514,2.560949,1.132196,...,-1.372687,1.020938,1.147131,-1.188418,0.987691,-0.403850,-0.469060,-1.500731,0.0,2.429243


In [10]:
target_coefs = df.T
target_coefs.columns = data.columns[2:]

In [12]:
target.to_csv('target_df.tsv', sep='\t')

In [19]:
for sample, sample_group in data.groupby('sample'):
    print(sample_group)

        sample  yield  sum_temp  mean_temp  sum_rain  mean_rain  sum_cloud  \
year                                                                         
2019  PS000026  115.0      90.7      18.14     218.5      43.70      29.36   
2020  PS000026  121.0      89.4      17.88     295.1      59.02      28.94   
2021  PS000026  117.0      91.3      18.26     335.5      67.10      29.80   
2022  PS000026  102.0      84.9      16.98     338.5      67.70      31.20   
2023  PS000026  108.0      89.3      17.86     344.4      68.88      26.70   

      mean_cloud  sum_humid  mean_humid  sum_sun  mean_sun  
year                                                        
2019       5.872      321.0        64.2   1465.0     293.0  
2020       5.788      287.0        57.4   1439.0     287.8  
2021       5.960      322.0        64.4   1376.0     275.2  
2022       6.240      331.0        66.2   1306.0     261.2  
2023       5.340      324.0        64.8   1471.0     294.2  
        sample  yield  sum